In [2]:
import netStandalone
import os
import numpy as np
from schmidt import BaseElement
from utils import plotter as plt
import time

In [3]:
# Path de workspace
PATH = os.path.join(os.environ['USERPROFILE'], "Desktop/CE")

# Paso de integración para los productos escalares
step = 0.1
# Dominio de las funciones [-domaine,domaine]
domaine = 128
# Orden, para distribuir los bias se va dividiendo el dominio de dos en dos -> 2^order neuronas base
order = 5

In [4]:
# Genera las funciones
X = np.arange(-domaine,domaine,step)

yfunc = np.zeros(int(2*domaine/step))
yfunc = np.sin(np.arange(-domaine,domaine,step)/10)+1
myElement = BaseElement.myfunc(yfunc)

In [7]:
# Guarda las funciones en archivo
with open(PATH+"/_temporal_data.csv", "w") as file:
    file.write("//number of elements\n"+str(len(yfunc))+",\n\n")
    file.write("//output isze, input size\n"+str(1)+","+str(1)+",\n\n")
    file.write("//inputs\n")
    for x in X:
        file.write(str(x)+",")
    file.write("\n//outputs\n")
    for y in yfunc:
        file.write(str(y)+",")
    file.write("\n//labels\n")
    for y in yfunc:
        file.write(str(0)+",")

In [8]:
# Crea la base de proyeccion
myBase = BaseElement.base(step, domaine, order)

In [9]:
# Escribe una red con las funciones proyectadas
entradas = 1
npl = [myBase.max_index,1]
act = [netStandalone.RELU2, netStandalone.RELU2]
myBase.write_params(PATH+"/pyparams.csv",entradas,npl,[myElement])

### VITIATE

In [15]:
# Inicializa el manejador de redes
handler = netStandalone.handler(PATH)
ITERATIONS = 10000
handler.attr(netStandalone.ALPHA, 2.0)
handler.attr(netStandalone.ALPHA_DECAY, 0.001)
handler.attr(netStandalone.ERROR_THRESHOLD, 0.1)
handler.attr(netStandalone.BATCH_SIZE, len(X))

In [11]:
handler.instantiate("red_proyectada", netStandalone.CPU)
handler.set_active_net("red_proyectada")
handler.build_net_from_file("pyparams",netStandalone.RELOAD_FILE)

In [18]:
tic = time.perf_counter()
handler.run_gradient(file=PATH+"_temporal_data.csv", file_reload=netStandalone.RELOAD_FILE)
print("Tiempo proyectada: "+str((time.perf_counter()-tic)))

Tiempo proyectada: 8.66999998834217e-05


In [19]:
handler.instantiate("red_aleatoria", netStandalone.CPU)
handler.set_active_net("red_aleatoria")
handler.build_net_from_data(entradas,netStandalone.v_int(npl),netStandalone.v_int(act))

In [21]:
tic = time.perf_counter()
handler.run_gradient(file=PATH+"_temporal_data.csv", file_reload=netStandalone.REUSE_FILE)
print("Tiempo aleatoria: "+str((time.perf_counter()-tic)))

Tiempo aleatoria: 0.00015129999974305974


In [22]:
0.00015129999974305974/8.66999998834217e-05*100

174.50980385986193